# MRI and TSV preprocessing 

## Imports

The data will be processed using 

In [ ]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # can copy and deepcopy files so original file is untouched
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import nibabel
import skimage


import sys
sys.path.insert(0, '../') # path to functions
from cvasl import file_handler as fh # 
from cvasl import mold #
from cvasl import carve
from cvasl.file_handler import Config

## Load files

Use the config pathways for the different datasets

In [ ]:
config = Config()
root_mri_directory = config.get_directory('raw_data')

In [ ]:
tsv_pattern = os.path.join(root_mri_directory, '**/*.tsv')
tsv_files = glob.glob(tsv_pattern, recursive=True)


In [ ]:
dataframe_example = pd.read_csv(tsv_files[0], sep='\t')

In [ ]:
dataframe_example

In [ ]:
mri_pattern = os.path.join(root_mri_directory, '**/*.gz')
gz_files = glob.glob(mri_pattern, recursive=True)

In [ ]:
# a path to an mri brain .nii image:
t1_fn = gz_files[2]

t1_img = nibabel.load(t1_fn)
# now display aspects of it

In [ ]:
t1_img.shape

In [ ]:
t1_hdr = t1_img.header
print(t1_hdr)


In [ ]:
t1_hdr.keys()


In [ ]:
# this may tell us the dimensions, which here we should note are 64,64,36 for 3 dimensions
t1_hdr['dim']

In [ ]:
t1_data = t1_img.get_fdata()
t1_data


In [ ]:
x_slice = t1_data[9, :, :]
y_slice = t1_data[:, 19, :]
z_slice = t1_data[:, :, 9]

In [ ]:
slices = [x_slice, y_slice, z_slice]

fig, axes = plt.subplots(1, len(slices))
for i, slice in enumerate(slices):
    axes[i].imshow(slice.T, cmap="gray", origin="lower")

## Look into charecteristics of image files

In [ ]:
shape_list = []
name_list = []
dim_key_list = []
for file in gz_files:
    #name = gz_files
    loaded = nibabel.load(file)
    header_here = loaded.header
    dim = header_here['dim']
    shape = loaded.shape
    shape_list.append(shape)
    name_list.append(file)
    dim_key_list.append(header_here)
    

In [ ]:
#name_list

In [ ]:
df = pd.DataFrame(name_list, shape_list)
df.columns = ["name"]
df = df.reset_index()
df = df.rename(columns={'index': 'shape_of'})

In [ ]:
df["dims"] = dim_key_list


In [ ]:
len(df.shape_of[0])


In [ ]:
df["len_shape"] = df["shape_of"].apply(lambda x: len(x))
df['string'] =  df["len_shape"].apply(lambda x: str(x))
nf = df.sort_values(by=['string'])
nf


In [ ]:
df.len_shape.max()

OK so out images are either 3 or 4 dimensions, and if the 4th dimension is one we can just cut it off, I hope. but sometimes it's more than one. Let's see if it's the 1-2 elements in the dim key... but we need to asses that we should rewrite above entlyieffic. Nonetheless let's look at an example where this is confusing

In [ ]:

t1_fn = gz_files[7]
t1_img = nibabel.load(t1_fn)
t1_hdr = t1_img.header
print(t1_hdr['dim'], t1_img.shape)

In [ ]:
t1_data = t1_img.get_fdata()
cut_data = t1_data[:, :, :, 0]
x_slice = cut_data[9, :, :]
y_slice = cut_data[:, 19, :]
z_slice = cut_data[:, :, 9]

In [ ]:
slices = [x_slice, y_slice, z_slice]

fig, axes = plt.subplots(1, len(slices))
for i, slice in enumerate(slices):
    axes[i].imshow(slice.T, cmap="gray", origin="lower")

Did I cutthe wrong dimension, looks like it is possible

In [ ]:
t1_data = t1_img.get_fdata()
cut_data = t1_data[:, :, 0, :]
x_slice = cut_data[10, :, :]
y_slice = cut_data[:, 19, :]
z_slice = cut_data[:, :, 19]

In [ ]:
slices = [x_slice, y_slice, z_slice]

fig, axes = plt.subplots(1, len(slices))
for i, slice in enumerate(slices):
    axes[i].imshow(slice.T, cmap="gray", origin="lower")

A mystery...but time to work on a key through function anyways....

In [ ]:
t1_fn = gz_files[2]
t1_img = nibabel.load(t1_fn)
struct_arr = t1_img.get_fdata() # get a NumPy array, without all the metadata
plt.imshow(struct_arr[15])

In [ ]:
plt.imshow(struct_arr[15], aspect=0.5)

In [ ]:
struct_arr2 = struct_arr.T
plt.imshow(struct_arr2[15])

In [ ]:
# ntb c

In [ ]:
%matplotlib inline

mold.SliceViewer(cut_data)

## Try using different aspect ratios?